In [1]:
import pandas as pd
# from api_methods import *
from api_methods import get_chanel_stats, get_video_ids, get_video_comments

from googleapiclient.discovery import build
from IPython.display import JSON

from config.config import YOUTUBE_API_KEY

In [2]:
# Get credentials and create an API client
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

In [3]:
import requests
import time

In [4]:
def get_video_statistics(video_id, youtube_api_key):
    # collecting video statistics
    video_stats_url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+youtube_api_key
    video_stats_response = requests.get(video_stats_url).json()

    view_count = video_stats_response['items'][0]["statistics"]["viewCount"]
    like_count = video_stats_response['items'][0]["statistics"]["likeCount"]

    return view_count, like_count

In [5]:
def get_videos_from_channel(df, channel_id, youtube_api_key):
    # make API call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+youtube_api_key+"&channelId="+channel_id+"&part=snippet,id&order=date&maxResults=500"+pageToken
    
    response = requests.get(url).json()
    time.sleep(3)

    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            video_title = video['snippet']['title']
            video_id = video['id']['videoId']
            upload_date = video['snippet']['publishedAt']
            upload_date= str(upload_date).split("T")[0]

            view_count, like_count = get_video_statistics(video_id, youtube_api_key)

            #save in pandas df
            df = df._append({
                "video_id": video_id,
                "video_title": video_title,
                "upload_date": upload_date,
                "view_count": view_count,
                "like_count": like_count
            }, 
            ignore_index = True
            )
    return df

In [6]:
CHANNEL_ID = "UCLA_DiR1FfKNvjuUpBHmylQ"

df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count"])
youtube_data = get_videos_from_channel(df, CHANNEL_ID, YOUTUBE_API_KEY)

In [7]:
youtube_data.head()

,video_id,video_title,upload_date,view_count,like_count
0,WnLhpmoA_hU,Rollout of Our Experimental Supersonic X-Plane...,2024-01-13,88929,2720
1,1nXlbgjIoP4,Axiom Mission 3 Launches to the International ...,2024-01-12,0,295
2,KjCdGqgD9Bs,Rollout of the X-59 Quesst Supersonic Plane (O...,2024-01-12,664483,16554
3,_tdsia6EZY8,Artemis: Onward to the Moon,2024-01-09,224315,13019
4,BWwnpVk6Wq4,First US Commercial Moon Launch: Astrobotic Pe...,2024-01-08,412978,11118


In [11]:
# Channels to analyze
channel_ids = [
    "UCLA_DiR1FfKNvjuUpBHmylQ",
    # Can add more channels
]

In [12]:
channel_stats = get_chanel_stats(youtube, channel_ids)

In [13]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,NASA,11600000,994421893,5760,UULA_DiR1FfKNvjuUpBHmylQ


In [14]:
playlist_id = channel_stats.loc[channel_stats['channelName']== "NASA", "playlistId"].iloc[0]

In [15]:
video_idss = get_video_ids(youtube, playlist_id)

In [16]:
len(video_idss)

5765

In [17]:
video_idss[:5]

['WnLhpmoA_hU', '1nXlbgjIoP4', 'KjCdGqgD9Bs', 'eC3Fl7Bt4qQ', '_tdsia6EZY8']

In [27]:
video_ids = [
    "h0oclM1Yw2A",
]

In [28]:
# Get video comments
video_comments = get_video_comments(youtube, video_ids)
# video_comments to dataframe
video_comments_df = pd.DataFrame(video_comments)

In [29]:
video_comments_df.head()

,0
0,They could have gotten better views from the t...
1,Bon courage 🎉
2,They failed to mention the first female astron...
3,🎉
4,ERES UNA VIAJERA EN EL TIEMPO ESCUCHANDO SU M...


In [30]:
# # save to csv
# video_comments_df.to_csv("video_comments.csv", index=False)